## More on performance, the CPU & GPU, and composability

Julia is shipped with OpenBLAS by default, and many linalg operations routed to it; in particular gemm, or mul! in julia

In [1]:
] instantiate

In [2]:
using LinearAlgebra

In [3]:
# Let's start with a single threaded m=n=k=4000 matrix multiply:
BLAS.set_num_threads(1)
single_thread_gflops = peakflops(4000) / 10^9

45.14279022693338

In [4]:
# What are we actually on? Call system `lstopo` (There's also Hwloc.jl, of course)
run(`lstopo --of txt`);

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Machine (63GB)                                                                                                                                                                                                 │
│                                                                                                                                                                                                                │
│ ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐ │
│ │ Package P#0                                                                                                                                             

In [5]:
# Maybe someone at CSCS will be disappointed with these numbers
# since it's using OpenBLAS, not MKL. But then again, don't we want
# to live in a world where we use open software?!
BLAS.set_num_threads(12)
multi_gflops = peakflops(4000) / 10^9

325.81507911080297

In [6]:
# Clearly OpenBLAS is not state-of-the art on Intel hardware, it doesn't scale well with more cores :(

round(multi_gflops / single_thread_gflops, digits=2)

7.22

### Pure Julia GEMM for the CPU

Can we get anything better than OpenBLAS? Yes! In fact we can get:

- any BLAS (license permitting...) through the BinaryBuilder project (MKL, BLIS, ...)
- a GEMM we're writing ourselves in Julia -- but standing on the shoulders of a loop vectorization package written in ... Julia

As soon as Julia 1.7 is release, [libblastrampoline](https://github.com/staticfloat/libblastrampoline) will be the default. It's light wrapper for BLAS that dispatches to whatever you have installed; very convenient! Also in the C, C++ and Python (?) world!

NOTE! If you want to use Julia's builtin threading (just threads or task-based parallellism), make sure it is actually enabled:

In [7]:
# I've created this by hand to make my life easier when doing threaded Julia code -- maybe restart your kernel if you haven't added this already
`cat $(ENV["HOME"])/.jupyterhub.env` |> run;

export JULIA_NUM_THREADS=12
export JULIA_PKG_DEVDIR="$SCRATCH/julia_dev"


In [8]:
Threads.nthreads()

12

Let's try out MKL first and see if we get to the peakflops

In [9]:
using BLASBenchmarksCPU: gemmmkl!, mkl_set_num_threads

In [10]:
mkl_set_num_threads(12)

In [11]:
function peakgflops(f!, n)
    A = rand(n, n); B = rand(n, n); C = rand(n, n)
    sec = @elapsed f!(C, A, B)
    return 2n^3 / sec / 10^9
end

peakgflops (generic function with 1 method)

In [12]:
peakgflops(gemmmkl!, 8_000)

375.6812956449564

In [14]:
using LoopVectorization

"""
Textbook matrix multiplication

The @avxt macro is used to enable LoopVectorization's ... loop vectorization
which supports nested loops. Note that this is generally unsafe! No bounds
checks and reordering of operations are allowed.
"""
function mygemm!(C, A, B)
    @avxt for m ∈ axes(A,1), n ∈ axes(B,2)
        Cmn = zero(eltype(C))
        for k ∈ axes(A,2)
           Cmn += A[m,k] * B[k,n]
        end
        C[m,n] = Cmn
    end
    
    return C
end


mygemm!

In [15]:
A = rand(10, 10)
B = rand(10, 10)
C = rand(10, 10)

using Test

# make sure not to reuse an existing output array in the test :D; so copy(A)
function verify(f!, A, B)
    result = f!(copy(A), A, B)
    expected = A * B
    norm(result - expected, Inf) < 10eps()
end

@test(verify(mygemm!, A, B)), @test(verify(gemmmkl!, A, B))

(Test Passed, Test Passed)

In [16]:
using BenchmarkTools, CUDA, UnicodePlots

function small_matrix_bench(n, ::Type{T} = Float64, peak = 2.6e9 #=Hz=# * 12 #=cores=# * 4 #=Float64/register=# * 4 #=units=#) where {T}
    # Allocate some matrices (cpu, gpu)
    A = rand(T, n, n); B = rand(T, n, n); C = rand(T, n, n)
    A_d = CUDA.rand(T, n, n); B_d = CUDA.rand(T, n, n); C_d = CUDA.rand(T, n, n)
    
    # Run benchmarks (will do warmup etc etc too)
    julia    = @belapsed mygemm!($C, $A, $B)
    mkl      = @belapsed gemmmkl!($C, $A, $B)
    cuda     = @belapsed CUDA.@sync mul!($C_d, $A_d, $B_d)
    
    # Show results
    julia_perc_of_peak = round((2n^3 / julia) / peak * 100, digits=2)
    mkl_perc_of_peak   = round((2n^3 / mkl)   / peak * 100, digits=2)
    cuda_perc_of_peak  = round((2n^3 / cuda)  / peak * 100, digits=2)
    
    println(barplot(
        ["Julia", "MKL", "cuBLAS"],
        [julia_perc_of_peak, mkl_perc_of_peak, cuda_perc_of_peak],
        title="Size $n x $n",
        xlabel="% of peak CPU performance"
    ), "\n")
    
    return nothing
end

small_matrix_bench (generic function with 3 methods)

In [38]:
small_matrix_bench(32)

                        Size 32 x 32
          ┌                                        ┐ 
    Julia ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 15.09   
      MKL ┤■■■■■■■■■■■■■■■ 6.97                      
   cuBLAS ┤■■ 0.82                                   
          └                                        ┘ 
                  % of peak CPU performance



In [39]:
# Let's test some "awkward" sizes for vectorization -- found in real life too
foreach(small_matrix_bench, 33:32:225)

                        Size 33 x 33
          ┌                                        ┐ 
    Julia ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 13.52   
      MKL ┤■■■■■■■■■■■■ 4.82                         
   cuBLAS ┤■■ 0.8                                    
          └                                        ┘ 
                  % of peak CPU performance

                        Size 65 x 65
          ┌                                        ┐ 
    Julia ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 31.63   
      MKL ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 29.53     
   cuBLAS ┤■■■■■ 5.27                                
          └                                        ┘ 
                  % of peak CPU performance

                        Size 97 x 97
          ┌                                        ┐ 
    Julia ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 60.57   
      MKL ┤■■■■■■■■■■■■■■■■■■■■■ 39.3                
   cuBLAS ┤■■■■■■■■ 15.0                             
          └                                

If this sparks your interest, read these resources, they are really nice with many plots: https://juliasimd.github.io/LoopVectorization.jl/stable/

## How about CUDA?

Clearly at modest sizes already the GPU is where you want to run your computations. But GPU programming is hard, right?

Julia makes it incredibly easy to write generic CUDA kernels. CUDA.jl is the most mature GPU programming package -- AMDGPU.jl is out there too, and they share a lot of code; device-agnostic, generic GPU programming is getting there!

### Writing a generic CUDA kernel that ±saturates the GPU

In [40]:
# ] add CUDA -- this is the package you want

In [41]:
using CUDA

"""
Dummy kernel doing 33 * 6 + 3 = 201 FMAs.
Should get us reasonably close to peak performance
Adapted from Tim Besard's example in CUDA.jl
"""
function kernel_201_fma(a, b, c, d, e, f, out)
    # side-note: the types of the kernel args will be CuDeviceArray, which
    # is multidimensional array type supporting cartesian indexing;
    # but are doing scalar operations, so stick to fast linear indexing.

    i = (blockIdx().x-1) * blockDim().x + threadIdx().x

    @inbounds if i ≤ length(out)
        aᵢ, bᵢ, cᵢ = a[i], b[i], c[i]
        dᵢ, eᵢ, fᵢ = d[i], e[i], f[i]

        for j = 1:33
            aᵢ′ = fma(aᵢ, bᵢ, cᵢ)
            bᵢ′ = fma(dᵢ, eᵢ, fᵢ)
            cᵢ′ = fma(cᵢ, bᵢ, aᵢ)
            dᵢ′ = fma(fᵢ, eᵢ, dᵢ)
            eᵢ′ = fma(aᵢ, cᵢ, eᵢ)
            fᵢ′ = fma(bᵢ, dᵢ, fᵢ)
            
            aᵢ, bᵢ, cᵢ = aᵢ′, bᵢ′, cᵢ′
            dᵢ, eᵢ, fᵢ = dᵢ′, eᵢ′, fᵢ′
        end

        out₁ = fma(aᵢ, bᵢ, cᵢ)
        out₂ = fma(dᵢ, eᵢ, fᵢ)
        out[i] = fma(out₁, out₂, out₂)
    end
    
    return
end

"""
Some generic benchmarking for number type T, applying
the kernel to n×n matrices.
"""
function cudapeakflops(::Type{T} = Float64, n = 5000) where {T}
    dims = (n, n)

    # Create 6 input arrays with random data
    # For Float32 / Float64 we can generate random numbers
    # directly on the device.
    
    # For the C++ folks: in a language where the line
    # between compile & runtime is blurred, there is
    # no need for `if *constexpr* T == Float64` 🙃
    # On a side-node: you have the freedom to use
    # emoji's as variable names in generic CUDA kernels
    # when you think that improves readability!
    if T === Float64 || T === Float32
        d_a = CUDA.rand(T, dims)
        d_b = CUDA.rand(T, dims)
        d_c = CUDA.rand(T, dims)
        d_d = CUDA.rand(T, dims)
        d_e = CUDA.rand(T, dims)
        d_f = CUDA.rand(T, dims)
    else
        d_a = CuArray(rand(T, dims))
        d_b = CuArray(rand(T, dims))
        d_c = CuArray(rand(T, dims))
        d_d = CuArray(rand(T, dims))
        d_e = CuArray(rand(T, dims))
        d_f = CuArray(rand(T, dims))
    end

    d_out = similar(d_a)

    len = prod(dims)

    # Create a kernel
    kernel = @cuda launch=false kernel_201_fma(d_a, d_b, d_c, d_d, d_e, d_f, d_out)
    config = launch_configuration(kernel.fun)
    threads = Base.min(len, config.threads)
    blocks = cld(len, threads)
    
    # Warm-up
    kernel(d_a, d_b, d_c, d_d, d_e, d_f, d_out)
    synchronize()

    secs = CUDA.@elapsed begin
        kernel(d_a, d_b, d_c, d_d, d_e, d_f, d_out; threads=threads, blocks=blocks)
    end

    # 201 fma's per kernel, 1 fma = 2 flops
    flopcount = 2 * 201 * len

    return round(flopcount / secs / 1e9, digits=2)
end

cudapeakflops

In [42]:
println(cudapeakflops(Float64), " GFlops")

3740.67 GFlops


In [43]:
println(cudapeakflops(Float32), " GFlops")

6629.15 GFlops


In [44]:
using LinearAlgebra: mul!

function another_peakflops(::Type{T}, n) where {T}
    A = CUDA.rand(T, n, n)
    B = CUDA.rand(T, n, n)
    C = CUDA.rand(T, n, n)
    
    sec = CUDA.@elapsed mul!(C, A, B)
    
    round(2n^3 / sec / 1e9, digits=2)
end

another_peakflops (generic function with 1 method)

In [45]:
println(another_peakflops(Float64, 5000), " GFlops")

4293.17 GFlops


In [46]:
println(another_peakflops(Float32, 5000), " GFlops")

2636.76 GFlops


In [47]:
CUDA.@device_code_ptx cudapeakflops(Float64)

// PTX CompilerJob of kernel kernel_201_fma(CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}) for sm_60

//
// Generated by LLVM NVPTX Back-End
//

.version 6.3
.target sm_60
.address_size 64

	// .globl	_Z26julia_kernel_201_fma_1146713CuDeviceArrayI7Float64Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EE // -- Begin function _Z26julia_kernel_201_fma_1146713CuDeviceArrayI7Float64Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EE
                                        // @_Z26julia_kernel_201_fma_1146713CuDeviceArrayI7Float64Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EE
.visible .entry _Z26julia_kernel_201_fma_1146713CuDeviceArrayI7Float64Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1E

In [48]:
CUDA.@device_code_ptx cudapeakflops(Float32) # not unrolled :( if you want to unroll a loop by hand to be 100% sure it does what you want, take a look at Base.Cartesian.@nexprs, a neat macro!

// PTX CompilerJob of kernel kernel_201_fma(CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}) for sm_60

//
// Generated by LLVM NVPTX Back-End
//

.version 6.3
.target sm_60
.address_size 64

	// .globl	_Z26julia_kernel_201_fma_1156313CuDeviceArrayI7Float32Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EE // -- Begin function _Z26julia_kernel_201_fma_1156313CuDeviceArrayI7Float32Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EE
.global .align 1 .b8 $str[11] = {95, 95, 67, 85, 68, 65, 95, 70, 84, 90, 0};
                                        // @_Z26julia_kernel_201_fma_1156313CuDeviceArrayI7Float32Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EES_IS0_Li2ELi1EE
.visible .entry _Z26julia_kernel_201_fma_11563

## Plugging a CUDA-unaware package into our kernel

Remember MultiFloats? They are very nice. But the package doesn't seem to have `fma` implemented :(

In [49]:
using MultiFloats

In [50]:
fma(Float64x2(1.0), Float64x2(1.0), Float64x2(1.0))

2.0

Let's add a generic `fma` function for MultiFloats:

In [51]:
import Base: fma

fma(x::MultiFloat, y::MultiFloat, z::MultiFloat) = x * y + z

fma (generic function with 33 methods)

In [52]:
using Test

@test fma(Float64x2(2.0), Float64x2(3.0), Float64x2(4.0)) == Float64x2(10.0)

Test Passed

Let's run our cudapeakflops function with Float64x{N}!

In [53]:
println(cudapeakflops(Float64x2), " GMultiFlx2ops (Giga MultiFloat{Float64,2} operations per second)")

124.36 GMultiFlx2ops (Giga MultiFloat{Float64,2} operations per second)


Succes!

One MultiFloat fma is 2 MultiFloat operations, but how many flop is a MultiFloat-op? Let's find out by using the GFlops package -- in another notebook

We counted an fma as 2 flops, but for Float64x2 it is actually 36 flops!

In [54]:
println(138.44 / 2 * 36, " GFlops for Float64x2, or ", round((138.44 / 2 * 36) / 4000 * 100, digits=2), "% of GPU peak")

2491.92 GFlops for Float64x2, or 62.3% of GPU peak


## Another instance of "unexpected" composability

In [55]:
A = rand(Float64x8, 100, 100)
B = rand(Float64x8, 100, 100)

# Do note that there is *some* precision loss, eps(Float64x8) == 5.9091063153828709e-126; it's tabulated in the MultiFloats.jl readme!
A * B - Array(CuArray(A) * CuArray(B))

100×100 Matrix{MultiFloat{Float64, 8}}:
 -9.09793e-115   5.40507e-115   2.88325e-115  …  -2.01245e-114   1.45208e-114
  1.38352e-114   2.33765e-114  -8.85281e-115     -7.29677e-115  -4.08252e-115
  1.95259e-115  -6.36662e-115  -1.01853e-115     -2.39703e-115   5.68613e-116
  5.23776e-116   2.08698e-115  -1.87998e-114     -8.64719e-115  -1.30447e-115
 -1.72312e-115  -5.42238e-115  -6.09206e-115      6.31143e-116  -6.16511e-115
 -1.67153e-115   5.44232e-115   1.22109e-114  …   1.06451e-114  -8.37738e-117
  6.19677e-116   1.71323e-114  -1.04796e-114     -1.55155e-114  -9.65254e-115
  5.23525e-115  -4.29548e-115   8.64368e-116     -3.4625e-115   -4.61405e-115
  4.30137e-115  -1.50597e-115   2.63562e-115      5.51376e-115  -4.17732e-115
 -4.5678e-116   -2.45388e-115   6.11103e-116      3.09443e-115   5.32819e-115
  1.74628e-114  -3.53623e-115   4.45584e-115  …  -6.63469e-115   4.08957e-115
 -1.02859e-114   1.22689e-114   8.70212e-115     -7.99985e-115   2.67055e-115
  1.4013e-114    2.52936

In [59]:
using LinearAlgebra: mul!
A_d = CuArray(rand(Float64x8, 1000, 1000))
B_d = CuArray(rand(Float64x8, 1000, 1000))
C_d = similar(A_d)

@time CUDA.@sync mul!(C_d, A_d, B_d);

  2.911391 seconds (2.84 M allocations: 109.230 MiB, 0.79% gc time, 10.90% compilation time)


Can we go further? Things started breaking down for me after double x10; but do note that we're there in roughly $2 \times 10^3$ flop per multiflop areas

In [60]:
MultiFloats.use_standard_multifloat_arithmetic(10)

In [61]:
A, B = rand(Float64x{10}, 10, 10), rand(Float64x{10}, 10, 10) # exercise for the reader :)
A

10×10 Matrix{Float64x{10}}:
 0.581641   0.944599  0.144027   …  0.704115     0.857323  0.618346
 0.153017   0.661059  0.43499       0.0191004    0.524941  0.445183
 0.829292   0.398566  0.869847      0.000906595  0.711319  0.201016
 0.446489   0.567816  0.851343      0.530009     0.194725  0.192469
 0.25493    0.828104  0.691898      0.9868       0.861545  0.0556866
 0.0812726  0.772254  0.581123   …  0.788337     0.784821  0.17692
 0.242289   0.896374  0.0804072     0.786013     0.277007  0.0382205
 0.380922   0.998832  0.608276      0.916063     0.96271   0.330946
 0.288848   0.187473  0.285251      0.590867     0.126445  0.954407
 0.386847   0.614478  0.112511      0.959511     0.617191  0.953837